In [1]:
# pip install git2vec
from git2vec import loader

In [30]:
# using a random repo as an example
repo_name = "https://github.com/voynow/ESPN-fantasy-football-3"

# return a list of Document objects
repo_data = loader.load(repo_name)

# or return a string of all the raw text
raw_repo = loader.load(repo_name, return_str=True)

In [3]:
resp = loader.get_trending_repos()

for reop in resp["items"]:
    loader.load(repo["clone_url"])

In [9]:
resp['items'][0]

{'id': 646480766,
 'node_id': 'R_kgDOJoiDfg',
 'name': 'roop',
 'full_name': 's0md3v/roop',
 'private': False,
 'owner': {'login': 's0md3v',
  'id': 26716802,
  'node_id': 'MDQ6VXNlcjI2NzE2ODAy',
  'avatar_url': 'https://avatars.githubusercontent.com/u/26716802?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/s0md3v',
  'html_url': 'https://github.com/s0md3v',
  'followers_url': 'https://api.github.com/users/s0md3v/followers',
  'following_url': 'https://api.github.com/users/s0md3v/following{/other_user}',
  'gists_url': 'https://api.github.com/users/s0md3v/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/s0md3v/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/s0md3v/subscriptions',
  'organizations_url': 'https://api.github.com/users/s0md3v/orgs',
  'repos_url': 'https://api.github.com/users/s0md3v/repos',
  'events_url': 'https://api.github.com/users/s0md3v/events{/privacy}',
  'received_events_url': 'https://api.github.co